In [1]:
# !jupyter nbconvert --to script FC_network_with_LSH.ipynb

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import tensorflow as tf
print(tf.__version__)
import make_datasets_modified as make_ds
import pickle
from random import shuffle

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import backend as K

num_channels = 3
window_size = 40
num_vUnits = num_channels*window_size
num_KCs = 15*num_vUnits
num_ones = num_KCs*6 #total number of ones in matrix
p_one = num_ones/(num_vUnits*num_KCs)
p_WTA = 0.05

batch_size = 128
epochs = 10

/home/jonas/anaconda3/envs/TF19/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.9.0



### must define p_WTA and num_KCs in make_dataset so that
### map_fn=WTA_activations_from_values_and_indices_map_fn()
### can be used as a map funtion
### also features must be in the order values,indices,one_hot_labels


In [3]:
load_path = '/home/jonas/HDD/data/unwindowed/Fourier_no_emphasis_WTA_vals_and_inds_15xKCs_RS112_WTA005_shuffled/'

keys = ['values','indices','one_hot_label']
data_types = ['float32','int32','int32']

In [4]:
training_files = []
validation_files = []
for file in os.listdir(load_path):
    if 'dataset5' in file:
        validation_files.append(load_path+file)
    elif 'dataset10' in file:
        validation_files.append(load_path+file)
    elif 'dataset15' in file:
        validation_files.append(load_path+file)
    elif 'dataset20' in file:
        validation_files.append(load_path+file)
    elif 'dataset25' in file:
        validation_files.append(load_path+file)
    else:
        training_files.append(load_path+file)
print(len(training_files))
print(len(validation_files))

32
5


In [5]:
load_path = '/home/jonas/HDD/data/unwindowed/Fourier_no_emphasis_WTA_vals_and_inds_15xKCs_RS112_WTA005/'
validation_files = []
for file in os.listdir(load_path):
    if 'dataset5' in file:
        validation_files.append(load_path+file)
    elif 'dataset10' in file:
        validation_files.append(load_path+file)
    elif 'dataset15' in file:
        validation_files.append(load_path+file)
    elif 'dataset20' in file:
        validation_files.append(load_path+file)
    elif 'dataset25' in file:
        validation_files.append(load_path+file)
    else:
        pass
        
validation_files = sorted(validation_files)
print(len(validation_files))

5


In [6]:
num_labels = 5

file_names = tf.placeholder(tf.string, shape=[None])
train_dataset = make_ds.dataset_from_TFRecords(training_files,batch_size,keys,data_types,
                                               shuffle_buffer=250000,num_cores=8,
                                               parallel_reads=len(training_files))
train_dataset = train_dataset.map(map_func=make_ds.WTA_activations_from_values_and_indices_map_fn,num_parallel_calls=8)
train_dataset = train_dataset.repeat()

test_dataset = make_ds.dataset_from_TFRecords(validation_files,batch_size,keys,data_types,
                                              shuffle_buffer=0,num_cores=8,
                                              parallel_reads=len(validation_files))
test_dataset = test_dataset.map(map_func=make_ds.WTA_activations_from_values_and_indices_map_fn,num_parallel_calls=8)
test_dataset = test_dataset.repeat()

a = Input(shape=(num_KCs,))
o = Dense(units=num_labels,activation='softmax')(a)

model = Model(inputs=a,outputs=o)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#Train model:
summary_path = '/home/jonas/Documents/LSH/LSH_runs/only_softmax_layer/Fourier_no_window_val_fixed/'
callbacks = [ModelCheckpoint(summary_path+'Model.hdf5',save_best_only=True,verbose=1)]
history = model.fit(train_dataset,epochs=epochs,steps_per_epoch=10000,
                    validation_data=test_dataset,validation_steps=1500,
                    verbose=1,callbacks=callbacks)

pickle_out = open(summary_path+'History.pickle','wb')
pickle.dump(history.history,pickle_out)
pickle_out.close()
K.clear_session()

Epoch 1/10
10000/10000 [==============================] - 50s 5ms/step - loss: 1.3099 - acc: 0.4668 - val_loss: 1.4027 - val_acc: 0.4281

Epoch 00001: val_loss improved from inf to 1.40267, saving model to /home/jonas/Documents/LSH/LSH_runs/only_softmax_layer/Fourier_no_window_val_fixed/Model.hdf5
Epoch 2/10
10000/10000 [==============================] - 46s 5ms/step - loss: 1.2868 - acc: 0.4776 - val_loss: 1.3426 - val_acc: 0.4559

Epoch 00002: val_loss improved from 1.40267 to 1.34256, saving model to /home/jonas/Documents/LSH/LSH_runs/only_softmax_layer/Fourier_no_window_val_fixed/Model.hdf5
Epoch 3/10
10000/10000 [==============================] - 48s 5ms/step - loss: 1.2835 - acc: 0.4785 - val_loss: 1.3388 - val_acc: 0.4610

Epoch 00003: val_loss improved from 1.34256 to 1.33876, saving model to /home/jonas/Documents/LSH/LSH_runs/only_softmax_layer/Fourier_no_window_val_fixed/Model.hdf5
Epoch 4/10
10000/10000 [==============================] - 48s 5ms/step - loss: 1.2874 - acc: 0.

In [7]:
#     ############################# for multiple trail runs
#     num_channels = 3
#     window_size = 40
#     num_vUnits = num_channels*window_size
#     num_KCs = 15*num_vUnits
#     num_ones = num_KCs*6 #total number of ones in matrix
#     p_one = num_ones/(num_vUnits*num_KCs)
#     load_path = '/home/jonas/HDD/data/unwindowed/Fourier_WTA_vals_and_inds_15xKCs_RS111_WTA005/'
    
#     def flatten1(x):
#         '''
#         flatten tensor x
#         -> results in [sample_ch1,sample_ch2,sample_ch3,sample_ch1,sample_ch2,...]
#         '''
#         flat = tf.reshape(x,[-1])
#         return flat

#     def WTA_activations_from_values_and_indices_map_fn(values,indices,labels):
#         '''
#         same as WTA_activations_from_values_and_indices but for a dataset
#         '''
#         num_KCs = 15*3*40

#         num_activations = int(p_WTA*num_KCs)
#         batch_size = tf.shape(values)[0]
#         indices = tf.stack([tf.stack([tf.range(start=0,limit=batch_size) for i in range(num_activations)],axis=1),
#                             indices],axis=-1)
#         values = flatten1(values)
#         indices = tf.reshape(indices,[-1,2])
#         WTA_rec = tf.scatter_nd(indices,values,shape=[batch_size,num_KCs])
#         return WTA_rec,labels
    
#     training_files = []
#     validation_files = []
#     for file in os.listdir(load_path):
#         if 'dataset5' in file:
#             validation_files.append(load_path+file)
#         elif 'dataset10' in file:
#             validation_files.append(load_path+file)
#         elif 'dataset15' in file:
#             validation_files.append(load_path+file)
#         elif 'dataset20' in file:
#             validation_files.append(load_path+file)
#         elif 'dataset25' in file:
#             validation_files.append(load_path+file)
#         else:
#             training_files.append(load_path+file)
#     print(len(training_files))
#     print(len(validation_files))
#     #############################
#############################
# dataset = dataset.map(map_func=WTA_activations_from_values_and_indices_map_fn,num_parallel_calls=8)
#############################